Блок 1: Импорт библиотек и базовые классы
Здесь мы подключаем необходимые библиотеки Python и создаем фундаментальные классы для игры. Определяем дескриптор для ограничения характеристик, систему эффектов и класс предметов.

In [39]:
# Блок 1: Импорт библиотек и базовые классы
import abc
import random
from typing import List, Dict, Optional
from enum import Enum

class BoundedStat:
    def __init__(self, min_val: int = 0, max_val: int = 1000):
        self.min_val = min_val
        self.max_val = max_val

    def __set_name__(self, owner, name):
        self.name = f"_{name}"

    def __get__(self, instance, owner):
        return getattr(instance, self.name, 0)

    def __set__(self, instance, value):
        value = max(self.min_val, min(value, self.max_val))
        setattr(instance, self.name, value)

class Effect:
    def __init__(self, name: str, duration: int, effect_type: str, value: int = 0):
        self.name = name
        self.duration = duration
        self.effect_type = effect_type
        self.value = value

    def apply_effect(self, target):
        if self.effect_type == 'dot':
            target.hp -= self.value
            print(f"🔴 {target.name} получает {self.value} урона от {self.name}")
        elif self.effect_type == 'hot':
            target.hp = min(target.hp + self.value, target.max_hp)
            print(f"🟢 {target.name} восстанавливает {self.value} HP от {self.name}")

    def update(self) -> bool:
        self.duration -= 1
        return self.duration > 0

class Item:
    def __init__(self, name: str, item_type: str, value: int):
        self.name = name
        self.item_type = item_type
        self.value = value

    def use(self, target):
        if self.item_type == 'health_potion':
            heal_amount = min(target.hp + self.value, target.max_hp)
            target.hp = heal_amount
            print(f"🧪 {target.name} использует {self.name} и восстанавливает {self.value} HP!")
        elif self.item_type == 'mana_potion':
            mana_amount = min(target.mp + self.value, target.max_mp)
            target.mp = mana_amount
            print(f"🔵 {target.name} использует {self.name} и восстанавливает {self.value} MP!")

print("✅ Блок 1 загружен: Базовые классы готовы!")

✅ Блок 1 загружен: Базовые классы готовы!


Блок 2: Классы персонажей
Создаем базовый класс Human с основными характеристиками и абстрактный класс Character для всех игровых персонажей. Добавляем миксин для критических ударов и систему навыков.

In [40]:
# Блок 2: Классы персонажей
class Human:
    hp = BoundedStat(0, 1000)
    mp = BoundedStat(0, 500)
    strength = BoundedStat(1, 100)
    agility = BoundedStat(1, 100)
    intellect = BoundedStat(1, 100)

    def __init__(self, name: str, level: int = 1):
        self.name = name
        self.level = level
        self._hp = 100
        self._mp = 50
        self._strength = 10
        self._agility = 10
        self._intellect = 10
        self.effects = []
        self.inventory = []

    @property
    def is_alive(self) -> bool:
        return self.hp > 0

    @property
    def max_hp(self):
        return 100 + (self.level - 1) * 20

    @property
    def max_mp(self):
        return 50 + (self.level - 1) * 10

    def __str__(self):
        return f"{self.name} (Ур. {self.level}) - HP: {self.hp}/{self.max_hp}, MP: {self.mp}/{self.max_mp}"

class Character(Human, abc.ABC):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self.skills_cooldown = {}
        self.available_skills = []

    @abc.abstractmethod
    def basic_attack(self, target) -> int:
        pass

    @abc.abstractmethod
    def get_skills(self) -> List[Dict]:
        pass

    def use_skill(self, skill_index: int, target) -> int:
        skills = self.get_skills()
        if skill_index < 0 or skill_index >= len(skills):
            return 0

        skill = skills[skill_index]

        if self.mp < skill['mp_cost']:
            print(f"❌ Недостаточно MP! Нужно {skill['mp_cost']}, есть {self.mp}")
            return 0

        self.mp -= skill['mp_cost']
        result = skill['action'](self, target)

        print(f"🎯 {self.name} использует {skill['name']} на {target.name}!")
        return result

    def update_effects(self):
        remaining_effects = []
        for effect in self.effects:
            effect.apply_effect(self)
            if effect.update():
                remaining_effects.append(effect)
        self.effects = remaining_effects

class CritMixin:
    def calculate_crit(self, base_damage: int) -> tuple:
        crit_chance = self.agility / 200
        is_crit = random.random() < crit_chance
        damage = base_damage * 2 if is_crit else base_damage
        return damage, is_crit

print("✅ Блок 2 загружен: Классы персонажей готовы!")

✅ Блок 2 загружен: Классы персонажей готовы!


Блок 3: Классы героев
Реализуем конкретные классы героев: Воина, Мага и Лекаря с уникальными способностями. Каждый класс имеет специальные навыки, соответствующие своей роли в бою.

In [ ]:
# Блок 3: Классы героев
class Warrior(Character, CritMixin):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._strength = 20 + (level - 1) * 3
        self._agility = 15 + (level - 1) * 2
        self._hp = 150 + (level - 1) * 30
        self._mp = 30 + (level - 1) * 5

    def basic_attack(self, target) -> int:
        damage, is_crit = self.calculate_crit(self.strength)
        target.hp -= damage
        crit_text = " КРИТИЧЕСКИЙ УДАР!" if is_crit else ""
        print(f"⚔️ {self.name} атакует {target.name} и наносит {damage} урона!{crit_text}")
        return damage

    def get_skills(self) -> List[Dict]:
        return [
            {
                'name': 'Силовой удар',
                'mp_cost': 20,
                'action': lambda c, t: self._power_strike(t)
            },
            {
                'name': 'Размашистый удар',
                'mp_cost': 30,
                'action': lambda c, t: self._cleave_attack(t)
            }
        ]

    def _power_strike(self, target) -> int:
        damage, is_crit = self.calculate_crit(self.strength * 2)
        target.hp -= damage
        return damage

    def _cleave_attack(self, target) -> int:
        damage = self.strength * 1.5
        target.hp -= damage
        return damage

class Mage(Character):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._intellect = 25 + (level - 1) * 4
        self._mp = 100 + (level - 1) * 20
        self._hp = 80 + (level - 1) * 15

    def basic_attack(self, target) -> int:
        damage = self.intellect // 2
        target.hp -= damage
        print(f"🔮 {self.name} атакует {target.name} магией и наносит {damage} урона!")
        return damage

    def get_skills(self) -> List[Dict]:
        return [
            {
                'name': 'Огненный шар',
                'mp_cost': 30,
                'action': lambda c, t: self._fireball(t)
            },
            {
                'name': 'Ледяная стрела',
                'mp_cost': 25,
                'action': lambda c, t: self._frost_bolt(t)
            }
        ]

    def _fireball(self, target) -> int:
        damage = self.intellect * 2
        target.hp -= damage
        target.effects.append(Effect("Горение", 2, 'dot', self.intellect // 2))
        return damage

    def _frost_bolt(self, target) -> int:
        damage = self.intellect * 1.5
        target.hp -= damage
        return damage

class Healer(Character):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self._intellect = 20 + (level - 1) * 3
        self._mp = 80 + (level - 1) * 15
        self._hp = 100 + (level - 1) * 20

    def basic_attack(self, target) -> int:
        damage = self.intellect // 3
        target.hp -= damage
        print(f"✨ {self.name} атакует {target.name} и наносит {damage} урона!")
        return damage

    def get_skills(self) -> List[Dict]:
        return [
            {
                'name': 'Лечение',
                'mp_cost': 25,
                'action': lambda c, t: self._heal(t)
            },
            {
                'name': 'Массовое лечение',
                'mp_cost': 40,
                'action': lambda c, t: self._mass_heal(t)
            }
        ]

    def _heal(self, target) -> int:
        heal_amount = self.intellect * 2
        target.hp = min(target.hp + heal_amount, target.max_hp)
        print(f"💚 {self.name} лечит {target.name} на {heal_amount} HP!")
        return heal_amount

    def _mass_heal(self, target) -> int:
        heal_amount = self.intellect
        print(f"💚 {self.name} исцеляет всю пати на {heal_amount} HP!")
        return heal_amount

print("✅ Блок 3 загружен: Классы героев готовы!")

Блок 4: Класс босса и итератор ходов
Создаем умного босса, который меняет поведение в зависимости от оставшегося здоровья и уровня сложности. Реализуем систему порядка ходов, где более ловкие персонажи ходят первыми.

In [42]:
# Блок 4: Класс босса и итератор ходов (ИСПРАВЛЕННАЯ ВЕРСИЯ)
class Boss(Character):
    def __init__(self, name: str, level: int = 5, difficulty: str = "normal"):
        super().__init__(name, level)

        # Настройки в зависимости от сложности
        difficulty_multiplier = {
            "easy": 0.7,
            "normal": 1.0,
            "hard": 1.5
        }
        multiplier = difficulty_multiplier.get(difficulty, 1.0)

        # Устанавливаем характеристики с учетом сложности
        self._hp = int(400 * multiplier) + (level - 1) * 80
        self._strength = int(25 * multiplier) + (level - 1) * 4
        self._agility = int(8 * multiplier) + (level - 1) * 2
        self._intellect = int(12 * multiplier) + (level - 1) * 3
        self.phase = 1
        self.difficulty = difficulty

    def basic_attack(self, target) -> int:
        damage = self.strength
        target.hp -= damage
        print(f"👹 {self.name} атакует {target.name} и наносит {damage} урона!")
        return damage

    def get_skills(self) -> List[Dict]:
        self._update_phase()

        if self.phase == 1:
            return self._phase1_skills()
        elif self.phase == 2:
            return self._phase2_skills()
        else:
            return self._phase3_skills()

    def _update_phase(self):
        max_hp = self.max_hp
        if self.hp > max_hp * 0.7:
            self.phase = 1
        elif self.hp > max_hp * 0.4:
            self.phase = 2
        else:
            self.phase = 3

    def _phase1_skills(self):
        return [{
            'name': 'Мощный удар',
            'mp_cost': 20,
            'action': lambda c, t: self._powerful_strike(t)
        }]

    def _phase2_skills(self):
        return [{
            'name': 'Крик войны',
            'mp_cost': 30,
            'action': lambda c, t: self._war_cry(t)
        }]

    def _phase3_skills(self):
        return [{
            'name': 'Ярость',
            'mp_cost': 40,
            'action': lambda c, t: self._fury_attack(t)
        }]

    def _powerful_strike(self, target) -> int:
        damage = self.strength * 2
        target.hp -= damage
        print(f"💥 {self.name} использует МОЩНЫЙ УДАР по {target.name}!")
        return damage

    def _war_cry(self, target) -> int:
        damage = self.strength * 1.5
        print(f"📢 {self.name} издает БОЕВОЙ КРИК!")
        return damage

    def _fury_attack(self, target) -> int:
        damage = self.strength * 2.5
        target.hp -= damage
        print(f"😡 {self.name} в ЯРОСТИ атакует {target.name}!")
        return damage

class TurnOrder:
    def __init__(self, characters: List[Character]):
        self.characters = sorted(
            [char for char in characters if char.is_alive],
            key=lambda x: x.agility,
            reverse=True
        )
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if not self.characters:
            raise StopIteration

        if self.index >= len(self.characters):
            self.index = 0
            raise StopIteration

        character = self.characters[self.index]
        self.index += 1
        return character

print("✅ Блок 4 загружен: Босс и система ходов готовы!")

✅ Блок 4 загружен: Босс и система ходов готовы!


Блок 5: Система боя с ограничением в 3 раунда
Разрабатываем основную механику боя с пошаговой системой и ограничением в три раунда. Добавляем интерфейс для выбора действий игроком и автоматическое поведение босса.

In [ ]:
# Блок 5: Система боя с ограничением в 3 раунда
class Battle:
    def __init__(self, party: List[Character], boss: Boss):
        self.party = party
        self.boss = boss
        self.round = 1
        self.max_rounds = 3  # Максимум 3 раунда!

    def start_battle(self):
        print("=== 🏰 НАЧАЛО БОЯ ===")
        print(f"⚡ Максимум {self.max_rounds} раунда!")
        print(f"🎯 Сложность: {self.boss.difficulty.upper()}")
        print(f"👥 Пати из {len(self.party)} персонажей против {self.boss.name}!")

        self._give_starting_items()

        while self._is_battle_ongoing() and self.round <= self.max_rounds:
            print(f"\n--- 📊 Раунд {self.round} ---")
            self._show_status()
            self._execute_round()
            self.round += 1

        self._declare_winner()

    def _give_starting_items(self):
        for character in self.party:
            character.inventory.extend([
                Item("Зелье здоровья", "health_potion", 50),
                Item("Зелье маны", "mana_potion", 30)
            ])

    def _is_battle_ongoing(self) -> bool:
        return any(char.is_alive for char in self.party) and self.boss.is_alive

    def _show_status(self):
        print("\n--- Статус пати ---")
        for char in self.party:
            status = "✅" if char.is_alive else "💀"
            print(f"{status} {char}")

        print(f"\n--- Статус босса ---")
        boss_status = "✅" if self.boss.is_alive else "💀"
        print(f"{boss_status} {self.boss}")

        # Показываем оставшиеся раунды
        rounds_left = self.max_rounds - self.round + 1
        print(f"\n⏰ Осталось раундов: {rounds_left}")

    def _execute_round(self):
        all_combatants = self.party + [self.boss]
        turn_order = TurnOrder(all_combatants)

        for character in turn_order:
            if not self._is_battle_ongoing() or self.round > self.max_rounds:
                break

            character.update_effects()

            if character.is_alive:
                if character == self.boss:
                    self._boss_turn(character)
                else:
                    self._player_turn(character)

    def _player_turn(self, character):
        print(f"\n🎮 Ход {character.name}:")
        print("1. ⚔️ Базовая атака")
        print("2. 🎯 Использовать навык")
        print("3. 🧪 Использовать предмет")
        print("4. ⏭️ Пропустить ход")

        try:
            choice = int(input("Выберите действие (1-4): "))

            if choice == 1:
                character.basic_attack(self.boss)
            elif choice == 2:
                self._use_player_skill(character)
            elif choice == 3:
                self._use_item(character)
            elif choice == 4:
                print(f"{character.name} пропускает ход.")
            else:
                print("Неверный выбор, пропускаем ход.")

        except ValueError:
            print("Неверный ввод, пропускаем ход.")

    def _use_player_skill(self, character):
        skills = character.get_skills()
        print("\nДоступные навыки:")
        for i, skill in enumerate(skills, 1):
            print(f"{i}. {skill['name']} (MP: {skill['mp_cost']})")

        try:
            skill_choice = int(input("Выберите навык: ")) - 1
            character.use_skill(skill_choice, self.boss)
        except (ValueError, IndexError):
            print("Неверный выбор навыка.")

    def _use_item(self, character):
        if not character.inventory:
            print("❌ Инвентарь пуст!")
            return

        print("\nИнвентарь:")
        for i, item in enumerate(character.inventory, 1):
            print(f"{i}. {item.name}")

        try:
            item_choice = int(input("Выберите предмет: ")) - 1
            item = character.inventory.pop(item_choice)
            item.use(character)
        except (ValueError, IndexError):
            print("Неверный выбор предмета.")

    def _boss_turn(self, boss):
        print(f"\n👹 Ход {boss.name}:")

        alive_players = [char for char in self.party if char.is_alive]
        if not alive_players:
            return

        target = random.choice(alive_players)

        if random.random() < 0.8:
            boss.use_skill(0, target)
        else:
            boss.basic_attack(target)

    def _declare_winner(self):
        print("\n" + "="*50)

        if self.round > self.max_rounds:
            print("⏰ ВРЕМЯ ВЫШЛО! Бой завершен по истечении 3 раундов!")

            # Определяем победителя по оставшемуся HP
            party_total_hp = sum(char.hp for char in self.party if char.is_alive)
            boss_hp = self.boss.hp

            if party_total_hp > boss_hp:
                print("🎉 ПОБЕДА! Пати выстояла в битве!")
            elif party_total_hp < boss_hp:
                print("💀 ПОРАЖЕНИЕ! Босс оказался сильнее!")
            else:
                print("🤝 НИЧЬЯ! Обе стороны проявили себя достойно!")

        elif self.boss.is_alive:
            print(f"💀 ПОРАЖЕНИЕ! Босс {self.boss.name} победил!")
        else:
            print(f"🎉 ПОБЕДА! Пати победила босса {self.boss.name}!")

        print(f"📊 Бой длился {self.round - 1} раундов")

print("✅ Блок 5 загружен: Система боя готова!")

Блок 6: Создание игры и запуск
Создаем функции для формирования команды игрока и выбора уровня сложности боя. Запускаем основной игровой цикл с настройками, выбранными пользователем.

In [43]:
# Блок 6: Создание игры и запуск
def create_party():
    party = []
    character_classes = {
        '1': ('Воин', Warrior),
        '2': ('Маг', Mage),
        '3': ('Лекарь', Healer),
        'воин': ('Воин', Warrior),
        'маг': ('Маг', Mage),
        'лекарь': ('Лекарь', Healer)
    }

    print("🎮 Создание пати (3 персонажа)")

    for i in range(3):
        print(f"\nВыбор персонажа {i + 1}:")
        print("1. ⚔️ Воин (сильная атака, высокая защита)")
        print("2. 🔮 Маг (магический урон, эффекты)")
        print("3. 💚 Лекарь (лечение, поддержка)")

        choice = input("Выберите класс (1-3 или название): ").lower().strip()

        if choice in character_classes:
            name = input("Введите имя персонажа: ").strip()
            if not name:
                name = f"Герой_{i+1}"

            class_name, class_obj = character_classes[choice]
            character = class_obj(name)
            party.append(character)
            print(f"✅ Добавлен {class_name}: {name}")
        else:
            print("❌ Неверный выбор! Доступные варианты: 1,2,3 или 'воин','маг','лекарь'")
            i -= 1
            continue

    return party

def select_difficulty():
    print("\n🎯 Выбор уровня сложности:")
    print("1. 🔰 Легкий (босс слабее)")
    print("2. ⚡ Нормальный (сбалансированный бой)")
    print("3. 💀 Сложный (босс сильнее)")

    difficulties = {
        '1': 'easy',
        '2': 'normal',
        '3': 'hard',
        'легкий': 'easy',
        'нормальный': 'normal',
        'сложный': 'hard'
    }

    while True:
        choice = input("Выберите сложность (1-3 или название): ").lower().strip()
        if choice in difficulties:
            return difficulties[choice]
        else:
            print("❌ Неверный выбор! Попробуйте снова.")

def main():
    print("🐉 Пати против Босса 🐉")
    print("=" * 40)
    print("⚡ Бой длится максимум 3 раунда!")
    print("🎯 Выберите стратегию wisely!")

    # Выбор сложности
    difficulty = select_difficulty()

    # Создаем пати
    party = create_party()

    print(f"\n🎉 Ваша пати создана! Состав:")
    for i, char in enumerate(party, 1):
        print(f"{i}. {char}")

    # Создаем босса в зависимости от сложности
    boss_names = ["Дракон", "Демон", "Великий Змей", "Повелитель Тьмы"]
    boss_name = random.choice(boss_names)
    boss = Boss(boss_name, level=len(party), difficulty=difficulty)

    difficulty_icons = {"easy": "🔰", "normal": "⚡", "hard": "💀"}
    print(f"\n🐲 Ваш противник: {boss_name} {difficulty_icons[difficulty]}")
    print(f"📊 Сложность: {difficulty.upper()}")

    # Начинаем бой
    input("\nНажмите Enter чтобы начать бой...")
    battle = Battle(party, boss)
    battle.start_battle()

# Запуск игры
if __name__ == "__main__":
    main()

print("✅ Блок 6 загружен: Игра готова к запуску!")


🐉 Пати против Босса 🐉
⚡ Бой длится максимум 3 раунда!
🎯 Выберите стратегию wisely!

🎯 Выбор уровня сложности:
1. 🔰 Легкий (босс слабее)
2. ⚡ Нормальный (сбалансированный бой)
3. 💀 Сложный (босс сильнее)
Выберите сложность (1-3 или название): 1
🎮 Создание пати (3 персонажа)

Выбор персонажа 1:
1. ⚔️ Воин (сильная атака, высокая защита)
2. 🔮 Маг (магический урон, эффекты)
3. 💚 Лекарь (лечение, поддержка)
Выберите класс (1-3 или название): 1
Введите имя персонажа: аа
✅ Добавлен Воин: аа

Выбор персонажа 2:
1. ⚔️ Воин (сильная атака, высокая защита)
2. 🔮 Маг (магический урон, эффекты)
3. 💚 Лекарь (лечение, поддержка)
Выберите класс (1-3 или название): 1
Введите имя персонажа: ввв
✅ Добавлен Воин: ввв

Выбор персонажа 3:
1. ⚔️ Воин (сильная атака, высокая защита)
2. 🔮 Маг (магический урон, эффекты)
3. 💚 Лекарь (лечение, поддержка)
Выберите класс (1-3 или название): 1
Введите имя персонажа: ыыыы
✅ Добавлен Воин: ыыыы

🎉 Ваша пати создана! Состав:
1. аа (Ур. 1) - HP: 150/100, MP: 30/50
2. ввв